In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
import json

### Scrapping Google Play Market with selenium 

#### Getting game links to futher parse individual game info

In [2]:
def pg_get_games_links(url,genre=''):
    driver = webdriver.Chrome()
    driver.get(url+genre)
    scroll_pause = 1
    time.sleep(scroll_pause)   
    last_height = driver.execute_script("return document.body.scrollHeight")
    time.sleep(scroll_pause)
    # Handling loading of additional content after scrolling to the bottom
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        last_height = new_height
    # Getting game links from page elements
    elems = driver.find_elements(By.XPATH, "//a[@href]")
    # end adding them into set to avoid possible duplication
    game_links = set()
    for elem in elems:
        attr = elem.get_attribute("href")
        if "details?id" in attr:
            game_links.add(attr)
            
    driver.close()
    
    return game_links

In [3]:
genre = 'GAME_STRATEGY'

In [4]:
url = 'https://play.google.com/store/apps/category/'

In [5]:
pglinks = pg_get_games_links(url,genre)

In [6]:
len(pglinks)

163

Lets also query for *rts* and *real time strategy* to have more data

In [7]:
url2 = 'https://play.google.com/store/search?q=real+time+strategy&c=apps'
pglinks2 = pg_get_games_links(url2)

In [8]:
url3 = 'https://play.google.com/store/search?q=rts&c=apps'
pglinks3 = pg_get_games_links(url3)

In [9]:
len(pglinks2)

30

In [10]:
len(pglinks3)

30

#### Parsing each link to get needed information

In [11]:
def parse_link(link):
    driver.get(link)
    
    title = driver.find_element(By.TAG_NAME, 'h1').text
    
    info = driver.find_element(By.CLASS_NAME, 'w7Iutd').text.split()
    dic = dict(list(zip(info[1::2], info[::2])))
    score = dic.get('star')
    ratings = dic.get('reviews')
    installed = dic.get('Downloads')

    
    elems = driver.find_elements(By.CLASS_NAME, 'VfPpkd-vQzf8d')
    # Free games got "Install" text on the button instead of price
    price = '0' if elems[0].text == 'Install' else elems[0].text.split()[:-1]
    
    tags = []  
    for e in elems[2:]:
        e_t = e.text
        if e_t != 'See details':
            tags.append(e_t)
        else:
            break
    # Locating and clicking "More info" button    
    driver.find_elements(By.CSS_SELECTOR, '.google-material-icons.VfPpkd-kBDsod.W7A5Qb')[0].click()
    time.sleep(0.2)
    
    more_info = driver.find_elements(By.CLASS_NAME, 'reAt0')
    dev = more_info[-1].text
    release_date = more_info[-2].text
    return {
        'title': title,
        'score': score,
        'ratings': ratings,
        'installed': installed, 
        'price': price,
        'tags': tags,
        'release_date': release_date,
        'offered_by': dev
    }

In [12]:
pglinks.update(pglinks2, pglinks3) # Combining game links

In [13]:
len(pglinks)

200

In [14]:
PG_games_data = []
driver = webdriver.Chrome()

# Iterating over links set and scrapping them individually
for l in pglinks:
    PG_games_data.append(parse_link(l))
driver.close()

In [15]:
pg_df = pd.DataFrame(PG_games_data)
pg_df

,title,score,ratings,installed,price,tags,release_date,offered_by
0,Viking Saga 1: The Cursed Ring,4.0,5.79K,100K+,0,"[Simulation, Management, Casual, Single player...","Jan 20, 2014",Qumaron
1,Bid Wars Stars - Multiplayer,4.4,13.2K,1M+,0,"[Simulation, Casual, Multiplayer, Competitive ...","Aug 13, 2020",By Aliens
2,World War Armies: WW2 PvP RTS,4.3,36.8K,1M+,0,"[Strategy, Tactics, Casual, Multiplayer, Compe...","Jun 29, 2022","Hypemasters, Inc."
3,Art of War: Legions,4.3,885K,50M+,0,"[Strategy, Tactics, Single player, Stylized, L...","Oct 20, 2019",Fastone Games HK
4,Card Guardians: Rogue Deck RPG,4.5,28.5K,1M+,0,"[Card, Card battler, Casual, Single player, St...","Sep 27, 2021",Tapps Games - PT
...,...,...,...,...,...,...,...,...
195,Kingdom War TD Premium Offline,4.4,3.22K,100K+,"[UAH, 15.00]","[Strategy, Tower defense, Casual, Stylized]","Oct 1, 2022",Fansipan Limited
196,The Battle of Polytopia,4.4,219K,10M+,0,"[Strategy, 4X, Casual, Multiplayer, Competitiv...","Nov 30, 2016",Midjiwan AB
197,Warage,3.6,1.27K,100K+,0,"[Strategy, Casual, Single player, Stylized, Of...","Feb 12, 2020",Digital Garbage
198,Age of Apes,4.6,325K,10M+,0,"[Strategy, Build & battle, Multiplayer, Compet...","Jul 28, 2020",tap4fun


In [16]:
# Getting unique tags across dataframe to look for target tags related to graphics
pg_df['tags'].explode('tags').unique()

array(['Simulation', 'Management', 'Casual', 'Single player', 'Stylized',
       'Offline', 'Multiplayer', 'Competitive multiplayer', 'Strategy',
       'Tactics', 'Stylized-realistic', 'Low poly', 'Card',
       'Card battler', 'Tower defense', 'Play Pass', 'Pixelated',
       'Tycoon', 'Build & battle', 'See all reviews', 'Idle',
       '#8 top grossing strategy', 'Wargame', 'Anime',
       '#9 top free strategy', 'Abstract', 'City-building', 'Realistic',
       '4X', '#4 top grossing strategy', 'Role Playing',
       'Puzzle Role-playing', 'Physics-puzzle RPG',
       '#6 top paid strategy', 'Action', 'Platformer',
       '#2 top free strategy', 'Tactical', '#9 top grossing strategy',
       '#1 top paid strategy', '#1 top free strategy', 'Action-adventure',
       'Government', 'Action-strategy', '#4 top paid strategy',
       'Turn-based RPG', 'Life', 'Empire building', 'Survival',
       '#10 top free strategy', '#8 top free strategy', 'Vehicle', 'Car',
       '#5 top free strate

In [17]:
# Tags of interest
target_tags = ['Stylized', 'Stylized-realistic', 'Pixelated', 'Anime', 'Low poly', 'Cartoon', 'Realistic',]

In [18]:
# Querying for target dataframe
df_pg_target = pg_df[pg_df['tags'].apply(lambda x: any(tag in x for tag in target_tags))]
df_pg_target

,title,score,ratings,installed,price,tags,release_date,offered_by
0,Viking Saga 1: The Cursed Ring,4.0,5.79K,100K+,0,"[Simulation, Management, Casual, Single player...","Jan 20, 2014",Qumaron
1,Bid Wars Stars - Multiplayer,4.4,13.2K,1M+,0,"[Simulation, Casual, Multiplayer, Competitive ...","Aug 13, 2020",By Aliens
2,World War Armies: WW2 PvP RTS,4.3,36.8K,1M+,0,"[Strategy, Tactics, Casual, Multiplayer, Compe...","Jun 29, 2022","Hypemasters, Inc."
3,Art of War: Legions,4.3,885K,50M+,0,"[Strategy, Tactics, Single player, Stylized, L...","Oct 20, 2019",Fastone Games HK
4,Card Guardians: Rogue Deck RPG,4.5,28.5K,1M+,0,"[Card, Card battler, Casual, Single player, St...","Sep 27, 2021",Tapps Games - PT
...,...,...,...,...,...,...,...,...
195,Kingdom War TD Premium Offline,4.4,3.22K,100K+,"[UAH, 15.00]","[Strategy, Tower defense, Casual, Stylized]","Oct 1, 2022",Fansipan Limited
196,The Battle of Polytopia,4.4,219K,10M+,0,"[Strategy, 4X, Casual, Multiplayer, Competitiv...","Nov 30, 2016",Midjiwan AB
197,Warage,3.6,1.27K,100K+,0,"[Strategy, Casual, Single player, Stylized, Of...","Feb 12, 2020",Digital Garbage
198,Age of Apes,4.6,325K,10M+,0,"[Strategy, Build & battle, Multiplayer, Compet...","Jul 28, 2020",tap4fun


In [19]:
df_pg_target.dtypes

title           object
score           object
ratings         object
installed       object
price           object
tags            object
release_date    object
offered_by      object
dtype: object

In [20]:
# Convert string numbers with suffix to numeric type
def proc_suffix(val):
  # print(val)
    if not val:
        return
    rx = r'^([0-9\.]+)\s*([km]?)\+?$'
    suffixes = {'k': 1000, 'm': 1000000}
    
    match = re.match(rx, val.lower(), re.IGNORECASE)
    number = float(match.group(1))
    suffix = match.group(2)

    return number * suffixes.get(suffix) if suffix else number

In [21]:
proc_suffix('262k+')

262000.0

In [22]:
df_pg_target.loc[:, ['ratings', 'installed']] = df_pg_target.loc[:, ['ratings', 'installed']].applymap(proc_suffix)

C:\Users\kulturkamp\AppData\Local\Temp\ipykernel_1528\2301881222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pg_target.loc[:, ['ratings', 'installed']] = df_pg_target.loc[:, ['ratings', 'installed']].applymap(proc_suffix)


In [23]:
df_pg_target

,title,score,ratings,installed,price,tags,release_date,offered_by
0,Viking Saga 1: The Cursed Ring,4.0,5790.0,100000.0,0,"[Simulation, Management, Casual, Single player...","Jan 20, 2014",Qumaron
1,Bid Wars Stars - Multiplayer,4.4,13200.0,1000000.0,0,"[Simulation, Casual, Multiplayer, Competitive ...","Aug 13, 2020",By Aliens
2,World War Armies: WW2 PvP RTS,4.3,36800.0,1000000.0,0,"[Strategy, Tactics, Casual, Multiplayer, Compe...","Jun 29, 2022","Hypemasters, Inc."
3,Art of War: Legions,4.3,885000.0,50000000.0,0,"[Strategy, Tactics, Single player, Stylized, L...","Oct 20, 2019",Fastone Games HK
4,Card Guardians: Rogue Deck RPG,4.5,28500.0,1000000.0,0,"[Card, Card battler, Casual, Single player, St...","Sep 27, 2021",Tapps Games - PT
...,...,...,...,...,...,...,...,...
195,Kingdom War TD Premium Offline,4.4,3220.0,100000.0,"[UAH, 15.00]","[Strategy, Tower defense, Casual, Stylized]","Oct 1, 2022",Fansipan Limited
196,The Battle of Polytopia,4.4,219000.0,10000000.0,0,"[Strategy, 4X, Casual, Multiplayer, Competitiv...","Nov 30, 2016",Midjiwan AB
197,Warage,3.6,1270.0,100000.0,0,"[Strategy, Casual, Single player, Stylized, Of...","Feb 12, 2020",Digital Garbage
198,Age of Apes,4.6,325000.0,10000000.0,0,"[Strategy, Build & battle, Multiplayer, Compet...","Jul 28, 2020",tap4fun


In [25]:
df_pg_target.sort_values(by='installed',ascending=False).head(20)

,title,score,ratings,installed,price,tags,release_date,offered_by
128,Clash of Clans,4.5,60300000.0,500000000.0,0,"[#7 top grossing strategy, Strategy, Build & b...","Sep 30, 2013",Supercell
111,Lords Mobile: Kingdom Wars,4.3,8280000.0,100000000.0,0,"[#3 top grossing strategy, Strategy, 4X, Multi...","Mar 11, 2016",IGG.COM
167,Evony: The King's Return,3.7,601000.0,100000000.0,0,"[Strategy, Build & battle, Casual, Multiplayer...","Aug 10, 2016",TG Inc.
168,Clash Royale,4.0,36200000.0,100000000.0,0,"[Strategy, Tactics, Casual, Multiplayer, Compe...","Mar 1, 2016",Supercell
39,Mafia City,4.4,1320000.0,100000000.0,0,"[Strategy, Build & battle, Casual, Single play...","Mar 30, 2017",Phantix Games
101,World of Tanks Blitz - PVP MMO,4.1,4340000.0,100000000.0,0,"[#1 top grossing strategy, Action, Shooter, Ve...","Dec 3, 2014",Wargaming Group
42,Plants vs. Zombies™,4.4,5620000.0,100000000.0,0,"[#2 top free strategy, Strategy, Tactics, Sing...","Nov 13, 2014",ELECTRONIC ARTS
193,Vlogger Go Viral: Tuber Life,4.5,2230000.0,50000000.0,0,"[Simulation, Life, Single player, Stylized, Lo...","Nov 10, 2015",Tapps Games
3,Art of War: Legions,4.3,885000.0,50000000.0,0,"[Strategy, Tactics, Single player, Stylized, L...","Oct 20, 2019",Fastone Games HK
179,State of Survival: Zombie War,4.5,2260000.0,50000000.0,0,"[#6 top grossing strategy, Strategy, Build & b...","Aug 26, 2019",FunPlus International AG


In [26]:
df_pg_target.sort_values(by=['score', 'ratings', 'installed'], ascending=[False, True, False]).head(20)

,title,score,ratings,installed,price,tags,release_date,offered_by
23,Block Fortress: War,5.0,2070.0,10000.0,"[UAH, 54.99]","[Strategy, Wargame, Casual, Stylized, Pixelated]","Oct 16, 2014",Foursaken Media
150,Defense Zone 3 Ultra HD,4.9,24000.0,500000.0,"[UAH, 79.99]","[Strategy, Tower defense, Casual, Single playe...","Feb 11, 2017",ARTEM KOTOV
7,King of Defense Premium,4.8,7560.0,100000.0,"[UAH, 47.99]","[Strategy, Tower defense, Casual, Single playe...","Jul 1, 2020",GCenter
41,Rusted Warfare - RTS Strategy,4.8,13500.0,100000.0,"[UAH, 54.99]","[Strategy, Wargame, Casual, Multiplayer, Compe...","Dec 31, 2011",Corroding games
48,Video Game Tycoon idle clicker,4.8,32200.0,1000000.0,0,"[Simulation, Management, Tycoon, Casual, Singl...","Apr 29, 2017",Holy Cow Studio
91,Iron Marines: RTS offline Game,4.8,79100.0,1000000.0,0,"[Strategy, 4X, Single player, Stylized, Offlin...","Sep 13, 2017",Ironhide Games
135,Kingdom Rush Vengeance TD Game,4.8,84300.0,500000.0,"[UAH, 219.99]","[Strategy, Tower defense, Casual, Single playe...","Nov 21, 2018",Ironhide Games
20,Dino Bash: Dinosaur Battle,4.8,140000.0,5000000.0,0,"[Strategy, Tactics, Casual, Single player, Sty...","Oct 5, 2015",Tilting Point
79,Kingdom Rush Origins TD,4.8,150000.0,1000000.0,"[UAH, 134.99]","[#4 top paid strategy, Strategy, Tower defense...","Nov 20, 2014",Ironhide Games
35,Kingdom Rush Frontiers TD,4.8,180000.0,1000000.0,"[UAH, 89.99]","[#6 top paid strategy, Strategy, Tower defense...","Sep 25, 2013",Ironhide Games


#### Saving Google play market games data

In [27]:
df_pg_target.to_csv('pg_rts_target.csv', mode='w')

### Using SteamSpy api and Steam store api to get PC games info

#### Getting list of RTS games listed on Steam store and approximate number of owners from SteamSpy

In [28]:
import requests

In [29]:
# Making a request for all RTS-tagged games and getting response
url = 'https://steamspy.com/api.php'
params = {
        'request': 'tag', 
        'tag': 'RTS'
 }

response = requests.get(url, params=params)
data = response.json()

For some reason, despite steamspy api should return more info(including tags) it does not \
Therefore tags will be collected in a separate request, and all other info can be conveniently retrieved from steam store api



In [30]:
list(data.values())[0]

{'appid': 570,
 'name': 'Dota 2',
 'developer': 'Valve',
 'publisher': 'Valve',
 'score_rank': '',
 'positive': 1632112,
 'negative': 347406,
 'userscore': 0,
 'owners': '100,000,000 .. 200,000,000',
 'average_forever': 36832,
 'average_2weeks': 1535,
 'median_forever': 934,
 'median_2weeks': 828,
 'price': '0',
 'initialprice': '0',
 'discount': '0',
 'ccu': 547671}

In [31]:
# Getting game id and number of owners from respone
rows = []
for game_id, game_data in data.items():
    row = {
        'game_id': game_id,
        'owners': game_data.get('owners', None),
    }
    rows.append(row)

In [32]:
df_steamspy_rts = pd.DataFrame(rows, columns=['game_id', 'owners'])
df_steamspy_rts

,game_id,owners
0,570,"100,000,000 .. 200,000,000"
1,227940,"10,000,000 .. 20,000,000"
2,489520,"10,000,000 .. 20,000,000"
3,394360,"5,000,000 .. 10,000,000"
4,594570,"5,000,000 .. 10,000,000"
...,...,...
1493,2255140,"0 .. 20,000"
1494,1085940,"0 .. 20,000"
1495,2240200,"0 .. 20,000"
1496,1362480,"0 .. 20,000"


In [33]:
def get_tags(json_data):
    row = {
        'game_id': json_data.get('appid'),
        'tags': list(json_data.get('tags').keys())
    }
    return row

In [34]:
url = 'https://steamspy.com/api.php'
data = []
for idd in df_steamspy_rts['game_id']:
    # Requesting individual game`s info
    params = {
            'request': 'appdetails', 
            'appid': idd
     }
    response = requests.get(url, params=params)
    row = get_tags(response.json())
    data.append(row)

In [35]:
df_steamspy_tags = pd.DataFrame(data)
df_steamspy_tags

,game_id,tags
0,570,"[Free to Play, MOBA, Multiplayer, Strategy, e-..."
1,227940,"[Free to Play, World War II, FPS, Multiplayer,..."
2,489520,"[Real Time Tactics, RTS, Multiplayer, Co-op, C..."
3,394360,"[Strategy, World War II, Grand Strategy, War, ..."
4,594570,"[Strategy, Fantasy, Turn-Based Strategy, RTS, ..."
...,...,...
1493,2255140,"[Social Deduction, Party Game, Mystery, Strate..."
1494,1085940,"[Strategy, RTS, Action, Indie, Violent, Buildi..."
1495,2240200,"[Strategy, RTS, Tower Defense, Rogue-like, Rea..."
1496,1362480,"[Education, Science, RTS, Creature Collector, ..."


#### Requesting information about each game id from Steam store api

In [36]:
# Parsing individual game` responce
def parse_steam_responce(json_data):
    for game_id, game_data in json_data.items():
        if game_data['success']:
            game_data = game_data['data']
            row = {
                'game_id': game_id,
                'title': game_data.get('name', None),
                'genres': [genre.get('description', None) for genre in game_data.get('genres', [])],
                'release_date': game_data.get('release_date', {}).get('date', None),
                'metacritic_score': game_data.get('metacritic', {}).get('score', None),
                'categories': [category.get('description', None) for category in game_data.get('categories', [])],
                'price_usd': game_data.get('price_overview', {}).get('final', None)/100 if 'price_overview' in game_data else None,
                'developers': game_data.get('developers', []),
                'publishers': game_data.get('publishers', [])
            }
            return row
    return {'game_id': game_id}

In [37]:
url = 'https://store.steampowered.com/api/appdetails/'
l = len(df_steamspy_rts['game_id'])

In [38]:
from IPython.display import clear_output

**! Unfortunately, Steam store api has a limit of 200 requests per 5 minutes, so reqiuesting should be paused for 300 seconds every 200 requests**

In [39]:
data = []
count = 0
for idd in df_steamspy_rts['game_id']:
    # Requesting individual game`s info
    params = {
        'appids':  idd,
        'cc': 'us',  
    }
    response = requests.get(url, params=params)
    steam_game_data = response.json()
    # Parcing response
    row = parse_steam_responce(steam_game_data)
    data.append(row)
    
    count += 1
    clear_output(wait=True)
    print(f'requests made: {count}; requests remaining: {l-count}; progress: {(count*100)/l:.2f}%')
    # Waiting for 300 seconds every 200 requests
    if count%200 == 0:
        time.sleep(300)

requests made: 1498; requests remaining: 0; progress: 100.00%


In [40]:
len(data)

1498

In [41]:
len(df_steamspy_rts['game_id'])

1498

In [42]:
df_steam_info = pd.DataFrame(data)
df_steam_info

,game_id,title,genres,release_date,metacritic_score,categories,price_usd,developers,publishers
0,570,Dota 2,"[Action, Free to Play, Strategy]","Jul 9, 2013",90.0,"[Multi-player, Co-op, Steam Trading Cards, Ste...",NaN,[Valve],[Valve]
1,227940,Heroes & Generals,"[Action, Free to Play, Indie, Massively Multip...","Oct 18, 2016",NaN,"[Multi-player, MMO, PvP, Online PvP, Co-op, On...",NaN,[TLM Partners],[TLM Partners]
2,489520,Minion Masters,"[Action, Adventure, Free to Play, Indie, RPG, ...","May 24, 2019",NaN,"[Single-player, Multi-player, PvP, Online PvP,...",NaN,[BetaDwarf],[BetaDwarf]
3,394360,Hearts of Iron IV,"[Simulation, Strategy]","Jun 6, 2016",83.0,"[Single-player, Multi-player, Co-op, Cross-Pla...",11.99,[Paradox Development Studio],[Paradox Interactive]
4,594570,Total War: WARHAMMER II,"[Action, Strategy]","Sep 28, 2017",87.0,"[Single-player, Multi-player, PvP, Online PvP,...",59.99,"[CREATIVE ASSEMBLY, Feral Interactive (Mac), F...","[SEGA, Feral Interactive (Mac), Feral Interact..."
...,...,...,...,...,...,...,...,...,...
1493,2255140,Paranoia Party,"[Casual, Free to Play, Indie, Strategy]","Feb 11, 2023",NaN,"[Multi-player, PvP, Online PvP, Includes level...",NaN,[axilirate],[axilirate]
1494,1085940,Orders Of The Ruler,"[Action, Indie, Strategy]","Mar 27, 2023",NaN,"[Single-player, Multi-player, PvP, Online PvP]",34.99,[E2D2Works],[E2D2Works]
1495,2240200,Final Fort,"[Casual, Indie, Simulation, Strategy]","Feb 21, 2023",NaN,[Single-player],3.99,[CloudDreamStudio],[CloudDreamStudio]
1496,1362480,战地细胞（Battlefield Cell）,"[Indie, Simulation, Strategy]","Aug 4, 2022",NaN,[Single-player],1.99,[E-Cell],[E-Cell]


#### Merging dataframes to get full game info

In [43]:
df_steam_full = df_steam_info.merge(df_steamspy_rts, on = 'game_id')
df_steam_full

,game_id,title,genres,release_date,metacritic_score,categories,price_usd,developers,publishers,owners
0,570,Dota 2,"[Action, Free to Play, Strategy]","Jul 9, 2013",90.0,"[Multi-player, Co-op, Steam Trading Cards, Ste...",NaN,[Valve],[Valve],"100,000,000 .. 200,000,000"
1,227940,Heroes & Generals,"[Action, Free to Play, Indie, Massively Multip...","Oct 18, 2016",NaN,"[Multi-player, MMO, PvP, Online PvP, Co-op, On...",NaN,[TLM Partners],[TLM Partners],"10,000,000 .. 20,000,000"
2,489520,Minion Masters,"[Action, Adventure, Free to Play, Indie, RPG, ...","May 24, 2019",NaN,"[Single-player, Multi-player, PvP, Online PvP,...",NaN,[BetaDwarf],[BetaDwarf],"10,000,000 .. 20,000,000"
3,394360,Hearts of Iron IV,"[Simulation, Strategy]","Jun 6, 2016",83.0,"[Single-player, Multi-player, Co-op, Cross-Pla...",11.99,[Paradox Development Studio],[Paradox Interactive],"5,000,000 .. 10,000,000"
4,594570,Total War: WARHAMMER II,"[Action, Strategy]","Sep 28, 2017",87.0,"[Single-player, Multi-player, PvP, Online PvP,...",59.99,"[CREATIVE ASSEMBLY, Feral Interactive (Mac), F...","[SEGA, Feral Interactive (Mac), Feral Interact...","5,000,000 .. 10,000,000"
...,...,...,...,...,...,...,...,...,...,...
1493,2255140,Paranoia Party,"[Casual, Free to Play, Indie, Strategy]","Feb 11, 2023",NaN,"[Multi-player, PvP, Online PvP, Includes level...",NaN,[axilirate],[axilirate],"0 .. 20,000"
1494,1085940,Orders Of The Ruler,"[Action, Indie, Strategy]","Mar 27, 2023",NaN,"[Single-player, Multi-player, PvP, Online PvP]",34.99,[E2D2Works],[E2D2Works],"0 .. 20,000"
1495,2240200,Final Fort,"[Casual, Indie, Simulation, Strategy]","Feb 21, 2023",NaN,[Single-player],3.99,[CloudDreamStudio],[CloudDreamStudio],"0 .. 20,000"
1496,1362480,战地细胞（Battlefield Cell）,"[Indie, Simulation, Strategy]","Aug 4, 2022",NaN,[Single-player],1.99,[E-Cell],[E-Cell],"0 .. 20,000"


In [44]:
df_steam_full.loc[:, 'game_id'] = df_steam_full.loc[:, 'game_id'].astype(int)
df_steam_full= df_steam_full.merge(df_steamspy_tags, on='game_id')
df_steam_full

,game_id,title,genres,release_date,metacritic_score,categories,price_usd,developers,publishers,owners,tags
0,570,Dota 2,"[Action, Free to Play, Strategy]","Jul 9, 2013",90.0,"[Multi-player, Co-op, Steam Trading Cards, Ste...",NaN,[Valve],[Valve],"100,000,000 .. 200,000,000","[Free to Play, MOBA, Multiplayer, Strategy, e-..."
1,227940,Heroes & Generals,"[Action, Free to Play, Indie, Massively Multip...","Oct 18, 2016",NaN,"[Multi-player, MMO, PvP, Online PvP, Co-op, On...",NaN,[TLM Partners],[TLM Partners],"10,000,000 .. 20,000,000","[Free to Play, World War II, FPS, Multiplayer,..."
2,489520,Minion Masters,"[Action, Adventure, Free to Play, Indie, RPG, ...","May 24, 2019",NaN,"[Single-player, Multi-player, PvP, Online PvP,...",NaN,[BetaDwarf],[BetaDwarf],"10,000,000 .. 20,000,000","[Real Time Tactics, RTS, Multiplayer, Co-op, C..."
3,394360,Hearts of Iron IV,"[Simulation, Strategy]","Jun 6, 2016",83.0,"[Single-player, Multi-player, Co-op, Cross-Pla...",11.99,[Paradox Development Studio],[Paradox Interactive],"5,000,000 .. 10,000,000","[Strategy, World War II, Grand Strategy, War, ..."
4,594570,Total War: WARHAMMER II,"[Action, Strategy]","Sep 28, 2017",87.0,"[Single-player, Multi-player, PvP, Online PvP,...",59.99,"[CREATIVE ASSEMBLY, Feral Interactive (Mac), F...","[SEGA, Feral Interactive (Mac), Feral Interact...","5,000,000 .. 10,000,000","[Strategy, Fantasy, Turn-Based Strategy, RTS, ..."
...,...,...,...,...,...,...,...,...,...,...,...
1493,2255140,Paranoia Party,"[Casual, Free to Play, Indie, Strategy]","Feb 11, 2023",NaN,"[Multi-player, PvP, Online PvP, Includes level...",NaN,[axilirate],[axilirate],"0 .. 20,000","[Social Deduction, Party Game, Mystery, Strate..."
1494,1085940,Orders Of The Ruler,"[Action, Indie, Strategy]","Mar 27, 2023",NaN,"[Single-player, Multi-player, PvP, Online PvP]",34.99,[E2D2Works],[E2D2Works],"0 .. 20,000","[Strategy, RTS, Action, Indie, Violent, Buildi..."
1495,2240200,Final Fort,"[Casual, Indie, Simulation, Strategy]","Feb 21, 2023",NaN,[Single-player],3.99,[CloudDreamStudio],[CloudDreamStudio],"0 .. 20,000","[Strategy, RTS, Tower Defense, Rogue-like, Rea..."
1496,1362480,战地细胞（Battlefield Cell）,"[Indie, Simulation, Strategy]","Aug 4, 2022",NaN,[Single-player],1.99,[E-Cell],[E-Cell],"0 .. 20,000","[Education, Science, RTS, Creature Collector, ..."


In [45]:
df_steam_full['tags'].explode('tags').unique()

array(['Free to Play', 'MOBA', 'Multiplayer', 'Strategy', 'e-sports',
       'Team-Based', 'Competitive', 'Action', 'Online Co-Op', 'PvP',
       'Difficult', 'Co-op', 'RTS', 'Tower Defense', 'RPG', 'Fantasy',
       'Character Customization', 'Replay Value', 'Action RPG',
       'Simulation', 'World War II', 'FPS', 'War', 'Shooter',
       'First-Person', 'Massively Multiplayer', 'Tactical', 'Historical',
       'Open World', 'Atmospheric', 'Adventure', 'Indie', 'Early Access',
       'Real Time Tactics', 'Card Battler', 'Strategy RPG', 'Card Game',
       'PvE', 'Deckbuilding', 'Singleplayer', 'Turn-Based Tactics',
       'Mouse only', 'Stylized', 'Grand Strategy', 'Military',
       'Alternate History', 'Real-Time with Pause', 'Diplomacy',
       'Sandbox', 'Turn-Based Strategy', 'Games Workshop', 'Turn-Based',
       'Dark Fantasy', 'Story Rich', 'Dinosaurs', 'Great Soundtrack',
       'Warhammer 40K', 'Medieval', 'Classic', 'Base-Building',
       'City Builder', 'Resource Managem

In [55]:
steam_target_tags = ['Stylized', 'Realistic', 'Gothic', 'Cartoon', 'Anime', 'Minimalist']

In [56]:
df_steam_full['categories'].explode('categories').unique()

array(['Multi-player', 'Co-op', 'Steam Trading Cards', 'Steam Workshop',
       'SteamVR Collectibles', 'In-App Purchases',
       'Valve Anti-Cheat enabled', 'MMO', 'PvP', 'Online PvP',
       'Online Co-op', 'Stats', 'Single-player',
       'Cross-Platform Multiplayer', 'Steam Achievements',
       'Full controller support', 'Steam Cloud', 'Remote Play on Tablet',
       'LAN PvP', 'LAN Co-op', 'Captions available', 'Steam Leaderboards',
       'Includes level editor', 'Remote Play on Phone',
       'Remote Play on TV', 'Partial Controller Support',
       'Shared/Split Screen', 'Remote Play Together',
       'Commentary available', 'VR Supported', 'Steam Turn Notifications',
       'Includes Source SDK', 'Shared/Split Screen PvP',
       'Shared/Split Screen Co-op', nan, 'Tracked Controller Support',
       'VR Only', 'VR Support'], dtype=object)

In [57]:
# Creating columns with numeric lower and upper bounds of estimated number of owners
df_steam_full['owners_min'] = df_steam_full.loc[:, 'owners'].apply(lambda x: int(x.split(' .. ')[0].replace(',', '')))
df_steam_full['owners_max'] = df_steam_full.loc[:, 'owners'].apply(lambda x: int(x.split(' .. ')[1].replace(',', '')))

In [58]:
df_steam_full

,game_id,title,genres,release_date,metacritic_score,categories,price_usd,developers,publishers,owners,tags,owners_min,owners_max
0,570,Dota 2,"[Action, Free to Play, Strategy]","Jul 9, 2013",90.0,"[Multi-player, Co-op, Steam Trading Cards, Ste...",NaN,[Valve],[Valve],"100,000,000 .. 200,000,000","[Free to Play, MOBA, Multiplayer, Strategy, e-...",100000000,200000000
1,227940,Heroes & Generals,"[Action, Free to Play, Indie, Massively Multip...","Oct 18, 2016",NaN,"[Multi-player, MMO, PvP, Online PvP, Co-op, On...",NaN,[TLM Partners],[TLM Partners],"10,000,000 .. 20,000,000","[Free to Play, World War II, FPS, Multiplayer,...",10000000,20000000
2,489520,Minion Masters,"[Action, Adventure, Free to Play, Indie, RPG, ...","May 24, 2019",NaN,"[Single-player, Multi-player, PvP, Online PvP,...",NaN,[BetaDwarf],[BetaDwarf],"10,000,000 .. 20,000,000","[Real Time Tactics, RTS, Multiplayer, Co-op, C...",10000000,20000000
3,394360,Hearts of Iron IV,"[Simulation, Strategy]","Jun 6, 2016",83.0,"[Single-player, Multi-player, Co-op, Cross-Pla...",11.99,[Paradox Development Studio],[Paradox Interactive],"5,000,000 .. 10,000,000","[Strategy, World War II, Grand Strategy, War, ...",5000000,10000000
4,594570,Total War: WARHAMMER II,"[Action, Strategy]","Sep 28, 2017",87.0,"[Single-player, Multi-player, PvP, Online PvP,...",59.99,"[CREATIVE ASSEMBLY, Feral Interactive (Mac), F...","[SEGA, Feral Interactive (Mac), Feral Interact...","5,000,000 .. 10,000,000","[Strategy, Fantasy, Turn-Based Strategy, RTS, ...",5000000,10000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1493,2255140,Paranoia Party,"[Casual, Free to Play, Indie, Strategy]","Feb 11, 2023",NaN,"[Multi-player, PvP, Online PvP, Includes level...",NaN,[axilirate],[axilirate],"0 .. 20,000","[Social Deduction, Party Game, Mystery, Strate...",0,20000
1494,1085940,Orders Of The Ruler,"[Action, Indie, Strategy]","Mar 27, 2023",NaN,"[Single-player, Multi-player, PvP, Online PvP]",34.99,[E2D2Works],[E2D2Works],"0 .. 20,000","[Strategy, RTS, Action, Indie, Violent, Buildi...",0,20000
1495,2240200,Final Fort,"[Casual, Indie, Simulation, Strategy]","Feb 21, 2023",NaN,[Single-player],3.99,[CloudDreamStudio],[CloudDreamStudio],"0 .. 20,000","[Strategy, RTS, Tower Defense, Rogue-like, Rea...",0,20000
1496,1362480,战地细胞（Battlefield Cell）,"[Indie, Simulation, Strategy]","Aug 4, 2022",NaN,[Single-player],1.99,[E-Cell],[E-Cell],"0 .. 20,000","[Education, Science, RTS, Creature Collector, ...",0,20000


In [59]:
df_steam_target = df_steam_full[df_steam_full['tags'].apply(lambda x: any(val in x for val in steam_target_tags))]
df_steam_target

,game_id,title,genres,release_date,metacritic_score,categories,price_usd,developers,publishers,owners,tags,owners_min,owners_max
2,489520,Minion Masters,"[Action, Adventure, Free to Play, Indie, RPG, ...","May 24, 2019",NaN,"[Single-player, Multi-player, PvP, Online PvP,...",NaN,[BetaDwarf],[BetaDwarf],"10,000,000 .. 20,000,000","[Real Time Tactics, RTS, Multiplayer, Co-op, C...",10000000,20000000
14,201270,Total War: SHOGUN 2,[Strategy],"Mar 15, 2011",90.0,"[Single-player, Multi-player, PvP, Online PvP,...",29.99,"[CREATIVE ASSEMBLY, Feral Interactive (Mac), F...","[SEGA, Feral Interactive (Mac), Feral Interact...","2,000,000 .. 5,000,000","[Strategy, Historical, Turn-Based Strategy, RT...",2000000,5000000
20,34330,Total War: SHOGUN 2,[Strategy],"Mar 15, 2011",90.0,"[Single-player, Multi-player, PvP, Online PvP,...",29.99,"[CREATIVE ASSEMBLY, Feral Interactive (Mac), F...","[SEGA, Feral Interactive (Mac), Feral Interact...","2,000,000 .. 5,000,000","[Strategy, Historical, Turn-Based Strategy, RT...",2000000,5000000
28,225260,Brutal Legend,"[Action, Adventure, Strategy]","Feb 26, 2013",80.0,"[Single-player, Multi-player, PvP, Online PvP,...",14.99,[Double Fine Productions],[Double Fine Productions],"2,000,000 .. 5,000,000","[Action, Comedy, Adventure, Great Soundtrack, ...",2000000,5000000
30,17390,SPORE™,"[Action, Adventure, Casual, RPG, Simulation, S...","Dec 19, 2008",84.0,"[Single-player, Steam Trading Cards]",19.99,[Maxis™],[Electronic Arts],"2,000,000 .. 5,000,000","[God Game, Open World, Exploration, Sandbox, C...",2000000,5000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,1606610,Rescuer,"[Action, Adventure, Casual, Indie, Strategy]","Aug 6, 2021",NaN,"[Single-player, Steam Achievements, Steam Cloud]",9.99,[GamesBraz],[GamesBraz],"0 .. 20,000","[Adventure, Action, Casual, Strategy, Action-A...",0,20000
1485,2206490,Ruined Kingdom,"[Action, Casual, Indie, RPG, Strategy]","Feb 28, 2023",NaN,"[Single-player, Steam Achievements, Steam Cloud]",14.99,"[Absolute Power Game Studio, https://absolutep...",[Absolute Power Game Studio],"0 .. 20,000","[RPG, Strategy, Action, Board Game, Grand Stra...",0,20000
1491,2307910,Pi Fu Fighter,"[Casual, Indie, Strategy]","Mar 9, 2023",NaN,[Single-player],2.99,[GANG DAN],[GANG DAN],"0 .. 20,000","[Minimalist, Casual, Simulation, 2D Platformer...",0,20000
1494,1085940,Orders Of The Ruler,"[Action, Indie, Strategy]","Mar 27, 2023",NaN,"[Single-player, Multi-player, PvP, Online PvP]",34.99,[E2D2Works],[E2D2Works],"0 .. 20,000","[Strategy, RTS, Action, Indie, Violent, Buildi...",0,20000


In [60]:
df_steam_target.sort_values(by='owners_min', ascending=False).head(20)

,game_id,title,genres,release_date,metacritic_score,categories,price_usd,developers,publishers,owners,tags,owners_min,owners_max
2,489520,Minion Masters,"[Action, Adventure, Free to Play, Indie, RPG, ...","May 24, 2019",NaN,"[Single-player, Multi-player, PvP, Online PvP,...",NaN,[BetaDwarf],[BetaDwarf],"10,000,000 .. 20,000,000","[Real Time Tactics, RTS, Multiplayer, Co-op, C...",10000000,20000000
20,34330,Total War: SHOGUN 2,[Strategy],"Mar 15, 2011",90.0,"[Single-player, Multi-player, PvP, Online PvP,...",29.99,"[CREATIVE ASSEMBLY, Feral Interactive (Mac), F...","[SEGA, Feral Interactive (Mac), Feral Interact...","2,000,000 .. 5,000,000","[Strategy, Historical, Turn-Based Strategy, RT...",2000000,5000000
28,225260,Brutal Legend,"[Action, Adventure, Strategy]","Feb 26, 2013",80.0,"[Single-player, Multi-player, PvP, Online PvP,...",14.99,[Double Fine Productions],[Double Fine Productions],"2,000,000 .. 5,000,000","[Action, Comedy, Adventure, Great Soundtrack, ...",2000000,5000000
30,17390,SPORE™,"[Action, Adventure, Casual, RPG, Simulation, S...","Dec 19, 2008",84.0,"[Single-player, Steam Trading Cards]",19.99,[Maxis™],[Electronic Arts],"2,000,000 .. 5,000,000","[God Game, Open World, Exploration, Sandbox, C...",2000000,5000000
33,244450,Men of War: Assault Squad 2,"[Action, Simulation, Strategy]","May 15, 2014",68.0,"[Single-player, Multi-player, Co-op, Steam Ach...",29.99,[Digitalmindsoft],[Fulqrum Publishing],"2,000,000 .. 5,000,000","[Strategy, World War II, War, Simulation, RTS,...",2000000,5000000
14,201270,Total War: SHOGUN 2,[Strategy],"Mar 15, 2011",90.0,"[Single-player, Multi-player, PvP, Online PvP,...",29.99,"[CREATIVE ASSEMBLY, Feral Interactive (Mac), F...","[SEGA, Feral Interactive (Mac), Feral Interact...","2,000,000 .. 5,000,000","[Strategy, Historical, Turn-Based Strategy, RT...",2000000,5000000
58,251060,Wargame: Red Dragon,"[Indie, Simulation, Strategy]","Apr 17, 2014",78.0,"[Single-player, Multi-player, PvP, Online PvP,...",29.99,[Eugen Systems],[Eugen Systems],"1,000,000 .. 2,000,000","[Strategy, RTS, Military, Realistic, Wargame, ...",1000000,2000000
61,302670,Call to Arms,"[Action, Indie, Simulation, Strategy]","Apr 27, 2018",NaN,"[Single-player, Multi-player, PvP, Online PvP,...",14.99,[Digitalmindsoft],[Digitalmindsoft],"1,000,000 .. 2,000,000","[Strategy, Military, Action, Multiplayer, RTS,...",1000000,2000000
68,558100,Art of War: Red Tides,"[Violent, Free to Play, Indie, Strategy, Early...","Dec 22, 2016",NaN,"[Multi-player, PvP, Online PvP, Co-op, Online ...",NaN,[Game Science],[Game Science],"1,000,000 .. 2,000,000","[Free to Play, Strategy, Multiplayer, RTS, Ear...",1000000,2000000
70,568220,Lobotomy Corporation | Monster Management Simu...,"[Indie, Simulation]","Apr 9, 2018",NaN,[Single-player],24.99,[ProjectMoon],[ProjectMoon],"1,000,000 .. 2,000,000","[Management, Difficult, Simulation, Story Rich...",1000000,2000000


#### Saving Steam store game data

In [61]:
df_steam_full.to_csv('steam_rts_full.csv', mode='w')

In [62]:
df_steam_target.to_csv('steam_rts_target.csv', mode='w')